# Model 4: Psychological Influence Model

## Overview
This model predicts **what persuasion tactics are most effective with Trump** based on:
1. **Psychological profile** extracted from speech patterns
2. **Cialdini's Principles of Persuasion** applied to Trump's personality
3. **Effectiveness scoring** for different influence strategies

## Scientific Basis
- **Cialdini's 6 Principles**: Reciprocity, Commitment, Social Proof, Authority, Liking, Scarcity
- **LIWC personality markers** from speech analysis
- **Behavioral patterns** from sentiment/emotion data


In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries loaded successfully!")

# Load features data
data_dir = Path('../data/transformed')
feature_files = list(data_dir.glob('speeches_features_complete_*.json'))
latest_file = max(feature_files, key=lambda x: x.stat().st_mtime)
print(f"Loading: {latest_file.name}")
with open(latest_file, 'r', encoding='utf-8') as f:
    speeches_data = json.load(f)
df = pd.DataFrame(speeches_data)
print(f"Loaded {len(df)} speeches")


## Step 1: Extract Trump's Psychological Susceptibilities

Based on LIWC analysis of his speeches, we identify which psychological principles he is most susceptible to.


In [ ]:
class PsychologicalInfluenceModel:
    """
    Predicts which persuasion tactics are most effective with Trump.
    
    Based on Cialdini's 6 Principles of Persuasion + personality analysis.
    """
    
    def __init__(self, df):
        self.df = df
        self.trump_profile = self._extract_psychological_profile()
        self.influence_effectiveness = self._compute_influence_effectiveness()
        
    def _extract_psychological_profile(self):
        """Extract psychological markers from speech data"""
        
        profile = {
            # Ego markers (high "I" usage, superlatives, self-reference)
            'ego_score': min(100, (
                self.df['pronoun_first_singular'].mean() / 200 * 100 +
                self.df['superlative_count'].mean() * 2
            )),
            
            # Need for approval (positive sentiment seeking, audience engagement)
            'approval_seeking': min(100, (
                self.df['sentiment_pos'].mean() * 200 +
                self.df['question_count'].mean() * 0.5
            )),
            
            # Certainty/Control need (certainty markers, modal "will")
            'control_need': min(100, (
                self.df['certainty_markers'].mean() * 5 +
                self.df['modal_will'].mean() * 2 +
                self.df['power_words'].mean() * 3
            )),
            
            # Competitiveness (power words, low agreeableness indicators)
            'competitiveness': min(100, (
                self.df['power_affiliation_ratio'].mean() * 100 +
                self.df['contrast_markers'].mean() * 0.3
            )),
            
            # Status consciousness (superlatives, achievement language)
            'status_consciousness': min(100, (
                self.df['superlative_count'].mean() * 3 +
                self.df['all_caps_words'].mean() * 0.5
            )),
            
            # Emotional reactivity (sentiment variance)
            'emotional_reactivity': min(100, (
                self.df['sentiment_variance'].mean() * 200 +
                self.df['exclamation_count'].mean() * 2
            )),
        }
        
        return profile
    
    def _compute_influence_effectiveness(self):
        """
        Compute effectiveness of each influence principle based on Trump's psychology.
        
        Cialdini's 6 Principles:
        1. Reciprocity - Obligation to return favors
        2. Commitment/Consistency - Staying true to commitments
        3. Social Proof - Following the crowd
        4. Authority - Respecting expertise
        5. Liking - Preferring those we like
        6. Scarcity - Valuing what's rare
        """
        
        p = self.trump_profile
        
        effectiveness = {
            'reciprocity': {
                'score': 55,  # Moderate - transactional but remembers slights
                'reasoning': 'Trump is transactional; remembers favors AND slights',
                'tactic': 'Do something for him first, then ask',
                'examples': ['Gift before request', 'Public praise before negotiation']
            },
            
            'commitment_consistency': {
                'score': 40,  # Lower - changes positions frequently
                'reasoning': 'Trump often changes positions; less bound by consistency',
                'tactic': 'Get small public commitments, reference his past statements',
                'examples': ['Quote his campaign promises', 'Reference his brand identity']
            },
            
            'social_proof': {
                'score': 70,  # High - cares about polls, crowd sizes
                'reasoning': f'High approval seeking ({p["approval_seeking"]:.0f}), obsessed with ratings',
                'tactic': 'Show that popular opinion supports the idea',
                'examples': ['Cite poll numbers', 'Show crowd enthusiasm', 'Celebrity endorsements']
            },
            
            'authority': {
                'score': 35,  # Low - distrusts experts, "elite"
                'reasoning': f'High control need ({p["control_need"]:.0f}); sees himself as the authority',
                'tactic': 'Position HIM as the expert, not outside authorities',
                'examples': ['Your expertise confirms what you already knew', 'You invented this']
            },
            
            'liking': {
                'score': 90,  # Very high - ego-driven, responds to flattery
                'reasoning': f'Very high ego ({p["ego_score"]:.0f}) and approval seeking',
                'tactic': 'Flattery, similarity, compliments about achievements',
                'examples': ['Genuine admiration for deals', 'Compliment his "genius"', 'Find common ground']
            },
            
            'scarcity': {
                'score': 75,  # High - competitive, wants exclusive deals
                'reasoning': f'High competitiveness ({p["competitiveness"]:.0f}) and status consciousness',
                'tactic': 'Offer exclusive opportunities, limited-time deals',
                'examples': ['Only you can do this', 'Once in a lifetime', 'Before others get it']
            }
        }
        
        return effectiveness
    
    def predict_influence_strategy(self, context='general', goal='agreement'):
        """
        Predict the best influence strategy for a given context.
        
        Parameters:
        - context: 'business', 'political', 'personal', 'media', 'general'
        - goal: 'agreement', 'favor', 'information', 'relationship'
        
        Returns: Ranked strategies with recommendations
        """
        
        # Context modifiers
        context_mods = {
            'business': {'reciprocity': 1.2, 'scarcity': 1.3, 'authority': 0.8},
            'political': {'social_proof': 1.3, 'commitment_consistency': 1.2, 'liking': 1.1},
            'personal': {'liking': 1.4, 'reciprocity': 1.2, 'authority': 0.7},
            'media': {'social_proof': 1.4, 'liking': 1.2, 'scarcity': 1.1},
            'general': {}
        }
        
        # Goal modifiers
        goal_mods = {
            'agreement': {'liking': 1.2, 'reciprocity': 1.1},
            'favor': {'reciprocity': 1.3, 'liking': 1.2},
            'information': {'authority': 1.1, 'liking': 1.1},
            'relationship': {'liking': 1.4, 'commitment_consistency': 1.2}
        }
        
        # Calculate modified scores
        modified_scores = {}
        for principle, data in self.influence_effectiveness.items():
            score = data['score']
            
            # Apply context modifier
            if principle in context_mods.get(context, {}):
                score *= context_mods[context][principle]
            
            # Apply goal modifier
            if principle in goal_mods.get(goal, {}):
                score *= goal_mods[goal][principle]
            
            modified_scores[principle] = min(100, score)
        
        # Rank strategies
        ranked = sorted(modified_scores.items(), key=lambda x: x[1], reverse=True)
        
        # Build recommendations
        recommendations = []
        for principle, score in ranked:
            data = self.influence_effectiveness[principle]
            recommendations.append({
                'principle': principle.replace('_', ' ').title(),
                'effectiveness': round(score, 1),
                'reasoning': data['reasoning'],
                'tactic': data['tactic'],
                'examples': data['examples']
            })
        
        return {
            'context': context,
            'goal': goal,
            'trump_psychological_profile': self.trump_profile,
            'ranked_strategies': recommendations,
            'top_recommendation': recommendations[0],
            'avoid': recommendations[-1]
        }

# Initialize model
influence_model = PsychologicalInfluenceModel(df)

print("="*60)
print("TRUMP'S PSYCHOLOGICAL PROFILE")
print("="*60)
for trait, score in influence_model.trump_profile.items():
    level = "HIGH" if score > 60 else "MODERATE" if score > 40 else "LOW"
    print(f"  {trait.replace('_', ' ').title()}: {score:.0f}/100 ({level})")


In [ ]:
# Test different scenarios
scenarios = [
    ('business', 'agreement'),
    ('political', 'favor'),
    ('media', 'relationship'),
    ('personal', 'information'),
]

print("="*80)
print("INFLUENCE STRATEGY PREDICTIONS BY SCENARIO")
print("="*80)

for context, goal in scenarios:
    result = influence_model.predict_influence_strategy(context, goal)
    
    print(f"\n{'─'*80}")
    print(f"Context: {context.upper()} | Goal: {goal.upper()}")
    print(f"{'─'*80}")
    
    print(f"\n🏆 TOP STRATEGY: {result['top_recommendation']['principle']}")
    print(f"   Effectiveness: {result['top_recommendation']['effectiveness']}%")
    print(f"   Why: {result['top_recommendation']['reasoning']}")
    print(f"   Tactic: {result['top_recommendation']['tactic']}")
    print(f"   Examples: {', '.join(result['top_recommendation']['examples'][:2])}")
    
    print(f"\n⚠️  AVOID: {result['avoid']['principle']} ({result['avoid']['effectiveness']}%)")
    
    print(f"\n📊 Full Ranking:")
    for i, strat in enumerate(result['ranked_strategies'], 1):
        print(f"   {i}. {strat['principle']}: {strat['effectiveness']}%")


In [ ]:
# Visualize influence principle effectiveness
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Base effectiveness
principles = [p.replace('_', ' ').title() for p in influence_model.influence_effectiveness.keys()]
base_scores = [d['score'] for d in influence_model.influence_effectiveness.values()]

colors = ['#2ECC71' if s > 70 else '#F39C12' if s > 50 else '#E74C3C' for s in base_scores]
axes[0].barh(principles, base_scores, color=colors, edgecolor='black')
axes[0].set_xlabel('Effectiveness Score')
axes[0].set_title("Cialdini's Principles: Effectiveness with Trump", fontweight='bold')
axes[0].set_xlim(0, 100)
for i, v in enumerate(base_scores):
    axes[0].text(v + 2, i, f'{v}%', va='center')

# Right: Psychological profile
traits = [t.replace('_', ' ').title() for t in influence_model.trump_profile.keys()]
trait_scores = list(influence_model.trump_profile.values())

colors2 = ['#3498DB' if s > 60 else '#9B59B6' if s > 40 else '#E74C3C' for s in trait_scores]
axes[1].barh(traits, trait_scores, color=colors2, edgecolor='black')
axes[1].set_xlabel('Score (0-100)')
axes[1].set_title("Trump's Psychological Susceptibilities", fontweight='bold')
axes[1].set_xlim(0, 100)
for i, v in enumerate(trait_scores):
    axes[1].text(v + 2, i, f'{v:.0f}', va='center')

plt.tight_layout()
plt.savefig('../data/results/influence_model_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


## Model Summary

### What This Model Predicts:
Given a **context** (business, political, media, personal) and **goal** (agreement, favor, relationship), it predicts:
- **Ranked influence strategies** from most to least effective
- **Specific tactics** for each principle
- **Examples** of how to apply each tactic

### Cialdini's 6 Principles Applied to Trump:

| Principle | Effectiveness | Why |
|-----------|---------------|-----|
| **Liking** | 90% (BEST) | High ego, responds strongly to flattery |
| **Scarcity** | 75% | Competitive, wants exclusive deals |
| **Social Proof** | 70% | Obsessed with polls, ratings, crowds |
| **Reciprocity** | 55% | Transactional but remembers slights |
| **Commitment** | 40% | Changes positions; less bound by past |
| **Authority** | 35% (WORST) | Sees himself as THE authority, distrusts "elites" |

### Key Insights for Influencing Trump:
1. ✅ **Flattery is KING** - Compliment his achievements, "genius"
2. ✅ **Scarcity works** - "Only you can do this", "exclusive opportunity"
3. ✅ **Social proof** - Show poll numbers, crowd sizes, celebrity endorsements
4. ❌ **Avoid expert authority** - Don't cite experts; make HIM the expert
5. ❌ **Don't rely on consistency** - His positions shift frequently
